# Tutorial to generate Ha cubes from a simulated disk galaxy

This tutorial processes the outputs of a RHD simulation of an idealised disk galaxy in order to generate a few spectra, images, and data-cubes around the H-alpha line. It models both the line emission and the stellar continuum, and accounts for light propagation through the dusty ISM. 


### Warning

For disk galaxy simulation, there are stars in the initial conditions, let say **old stars**. Make sure to read them correctly as stars and not dark matter and to set their ages and metallicity, since they do contribute to the stellar continuum.


## 1. Some imports and useful functions

In [ ]:
RascasDir       = '../../../'  # path to the rascas directory (could be an absolute path).
import sys
sys.path.append("%s/py/"%(RascasDir))
# imports from the RASCAS library
import write_param_files as wpf
import jphot as jp
import lya_utils as lya
from mocks import mockobs

In [ ]:
# standard imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
from astropy import units as u

In [ ]:
def v_from_lambda(lbda_Angstrom):
    Halpha_lambda0 = 6562.8e-8 # [cm]
    Halpha_nu0     = lya.clight / Halpha_lambda0  # [Hz]
    nu = lya.clight / (lbda_Angstrom * 1e-8) # [Hz]
    v = Halpha_lambda0 * (Halpha_nu0 - nu) # [cm/s]
    return v*1e-5 # [km/s]

def lambda_from_v(v_kms):
    Halpha_lambda0 = 6562.8e-8 # [cm]
    lbda = Halpha_lambda0 / ( 1 - v_kms*1e5/lya.clight) # [cm]
    return lbda*1e8  # [A]

print(lambda_from_v(-400),lambda_from_v(400))

In [ ]:
# a useful function to get units from ramses outputs
def get_info_param_real(RamsesDir,timestep,pname):
    """ reads the value of a parameter from a ramses info file. 
    inputs: 
    - RamsesDir : where the directories output_xxxxx are found.
    - timestep : the snapshot number
    - pname : the name of the parameter to search for in the info file. 
    """
    file = RamsesDir+'/output_%5.5i'%timestep+'/info_%5.5i.txt'%timestep
    f = open(file,'r')
    lines = f.readlines()
    f.close()
    for l in lines:
        if pname in l:
            p = np.float64(l.split(sep='=')[1])
            return p


## 2. General setup
(Here we set up paths to ramses data and define a few general things)

In [ ]:
ExperimentDir   = './'  # where the RASCAS run will happen (and where param file will be written). 
repository = '/scratch/Cral/rosdahl/23_crmom/01_galdisks/02_G9/12_flike_nocr/'
snapnum    = 18
# properties of the region to compute RT in :
targetshape = 'sphere'
targetpos  = '0.5, 0.5, 0.5' 
targetrad  = 0.1

RASCAS computes the emission from star particles and gas cells within the target volume, and propagate Monte Carlo photon packets until the escape that volume. Different volumes are possible in principle (cube, slab, sphere, shell, ...) but for constructing mocks as we do here, the sphere is the most convenient. 

We thus assume the region to be a sphere (`targetshape`) and we define its radius via the variable `targetrad` (in **box units**). One may want to express the target radius in kpc... We do this below as an example.

In [ ]:
unit_l = get_info_param_real(repository,snapnum,'unit_l') # read unit_l from info file to convert box units to cm
boxlen = get_info_param_real(repository,snapnum,'boxlen')
targetrad_units = targetrad * unit_l * boxlen * u.cm
print(targetrad_units.to('kpc'))

## 3. Define mock observations
In the bloc below, we define the parameters for mock observations. These parameters include pointings (i.e. target position and directions of observations), and instrumental specifications (spectroscopy, imaging, cube properties). The resulting parameters are written to a user-unfriendly file `mock_param_file.txt` in the `ExperimentDir` directory. This file is later used by `rascas`.

The way things are written below are so that the cube has the same properties as the image and spectrum. This needs not be the case: all mocks in a given pointing are independent. 

In [ ]:
target_position = np.array([0.5,0.5,0.5])  # aim for the center of the box (all in box units here... )

# draw a few directions from edge-on to face-on
nmocks   = 5  # how many pointings
theta = np.linspace(0,np.pi/2.,nmocks)
kobs = []
for t in theta:
    kobs.append(np.array([0,np.cos(t),np.sin(t)]))  # observe along the y direction (make sure k is normalised)
    
# spectroscopic observation
spec_npix       = 50 # the nb of wavelength bins
aperture   = 0.03 # the radius of the circular aperture within which photon packets are collected (in box-size units)
lambda_min = 6554  # min wavelength (Angstrom)
lambda_max = 6571.6  # max wavelength (Angstrom)
spec       = wpf.specParams(spec_npix,aperture,lambda_min,lambda_max)

# image 
im_npix = 100 # nb of pixels on a side
size = 0.05   # size of the image (side)
image = wpf.imageParams(im_npix,size)

# cube 
cube_lbda_npix = spec_npix
cube_image_npix = im_npix
cube_lmin = lambda_min        # Angstrom
cube_lmax = lambda_max        # Angstrom
cube_side = size              # box units [0,1] (pixel_size = size/im_npix)
cube = wpf.cubeParams(cube_lbda_npix,cube_image_npix,cube_lmin,cube_lmax,cube_side)

# define the object containing all the above information for each direction
# and write it into some file 
mock_params_file = '%s/mock_param_file.txt'%ExperimentDir
f = open(mock_params_file,'w')
for k in kobs:
    p = wpf.pointing(k,target_position,spec=spec,image=image,cube=cube)
    # p = wpf.pointing(k,target_position,cube=cube)
    p.write(f)
f.close()


## 4. Run RASCAS suite of codes 

### 4.1 Generate photon packet initial conditions
Here we write parameter files and execute the codes `PhotonsFromStars` and `HaPhotonsFromGas`. These are serial codes which read the ramses outputs, compute the luminosities of star particles and gas cells and generate a samplic of these in the form of photon packets.

In [ ]:
Parameters = []  # initialise a list of parameter sections


# Stellar continuum 
section = 'PhotonsFromStars'
params = {
    'repository'          : repository,
    'snapnum'             : '%i'%snapnum ,
    'outputfile'          : '%s/Cont.IC'%ExperimentDir,
    'star_dom_type'       : targetshape ,
    'star_dom_pos'        : targetpos,
    'star_dom_rsp'        : '%.8e'%targetrad ,
    'spec_SSPdir'         : '/Xnfs/cosmos/lmdansac/benchmark_PSMN/SED/bpass_v221/bpass_v221_chab100/',
    'spec_type'           : 'Table' ,
    'spec_table_lmin_Ang' : '%f'%lambda_min,
    'spec_table_lmax_Ang' : '%f'%lambda_max,
    'nPhotonPackets'      : '500000'
}
Parameters.append(wpf.param_section(section,params))

# H-alpha emission (collisions and recombinations)
section = 'HaPhotonsFromGas'
params = {
    'outputfileRec'     : '%s/RecHa.IC'%ExperimentDir,
    'outputfileCol'     : '%s/ColHa.IC'%ExperimentDir,
    'repository'        : repository,
    'snapnum'           : '%i'%snapnum,
    'emission_dom_type' : targetshape,
    'emission_dom_pos'  : targetpos,
    'emission_dom_rsp'  : '%.8e'%targetrad,
    'nPhotonPackets'    : '100000 ',
    'ranseed'           : '-111',
    'verbose'           : 'T ',
    'doRecombs'         : 'T ',
    'doColls'           : 'T',
    'tcool_resolution'  :'0'
}
Parameters.append(wpf.param_section(section,params))

# RAMSES details -> WARNING: check that the indices of variables match the simulation you process
# (find the information in output_xxxxx/hydro_file_descriptor.txt)
section_ramses = 'ramses'
params_ramses = {
    'self_shielding'    : 'T ',
    'ramses_rt'         : 'T ',
    'read_rt_variables' : 'T ',
    'verbose'           : 'T',
    'cosmo'             : 'F',
    'use_proper_time'   : 'T',
    'particle_families' : 'T',
    'itemp'             : '11',
    'imetal'            : '12',
    'ihii'              : '13',
    'iheii'             : '14',
    'iheiii'            : '15',
    'deut2H_nb_ratio'   : '3.0e-05',
    'recompute_particle_initial_mass' : 'F'
}
Parameters.append(wpf.param_section(section_ramses,params_ramses))

#### write parameter list to a file 
wpf.write_parameter_file('%s/PPIC.cfg'%(ExperimentDir),Parameters)

In [ ]:
cmd = "%s/f90/PhotonsFromStars %s/PPIC.cfg > %s/PhotonsFromStars.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file PhotonsFromStars.log or message below for more info')

cmd = "%s/f90/HaPhotonsFromGas %s/PPIC.cfg > %s/HaPhotonsFromGas.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file HaPhotonsFromGas.log or message below for more info')
    

### 4.2 Generate mesh for RT
Here we write the parameter file and run the code `CreateDomDump` which reads ramses outputs and extracts the mesh in the region of interest, with the quantities of interest. 

In RASCAS the dust model follows the formulation of Laursen et al (2009):
$$ n_{dust} = (n_{HI} + f_{ion}\ n_{HII}) \frac{Z}{Z_0} $$

with $f_{ion} ∼ 0.01$ a free parameter describing how much dust is present in ionised gas,
and $Z_0 ∼ 0.005 (0.01)$ the mean metallicity of the Small and Large Magellanic Cloud (SMC and LMC, respectively).

In [ ]:
Parameters = []  # initialise a list of parameter sections

# Gas and dust
section = 'CreateDomDump'
params = {
    'repository'       : repository,
    'snapnum'          : '%i'%snapnum ,
    'DomDumpDir'       : './',
    'idealised_models' : 'F',
    'comput_dom_type'  : targetshape,
    'comput_dom_pos'   : targetpos,
    'comput_dom_rsp'   : '%.8e'%targetrad
}
Parameters.append(wpf.param_section(section,params))

# H-alpha does not scatter ... use only dust. 
section = 'gas_composition'
params = {
    'nscatterer' : '0',
    'f_ion'      : '1.000E-02',
    'Zref'       : '5.000E-03',
    'ignoreDust' : 'F'
}
Parameters.append(wpf.param_section(section,params))

# RAMSES details
# same as above
Parameters.append(wpf.param_section(section_ramses,params_ramses))

#### write parameter list to a file 
wpf.write_parameter_file('%s/CDD.cfg'%(ExperimentDir),Parameters)

In [ ]:
cmd = "%s/f90/CreateDomDump %s/CDD.cfg > %s/CreateDomDump.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file CreateDomDump.log or message below for more info')


Quick visualization of the mesh 

In [ ]:
# visualise the mesh

# Short note:
# AMR mesh means that cells have different sizes. Here, we want to visualise the size of the cells. 
# We will make a map of the maximum level of refinement of the cells along the line-of-sight. 
# (Higher refinement level means more resolved, so smallest cells.)
# To better see how resolution changes with radius in the case of the sphere, 
# we will make a map by selecting only cells in a thin slice along the line-of-sight.

import maps as ma
import mesh as me

# read the mesh file
mdom = me.mesh(filename="domain_1.mesh",gasmix="dust")

lmax = np.amax(mdom.gas.leaflevel)
levels = np.array(mdom.gas.leaflevel[:])
positions = mdom.gas.xleaf

# make a map of the maximum level of refinement of the mesh
map, _ = ma.lmax_map(lmax,positions,levels,0.,1.,0.499,0.501,0.,1.,'y')

im = plt.imshow(map.T,interpolation='nearest',origin='lower',
                    extent=(0.,1.,0.,1.),
                    cmap='jet')
cbar = plt.colorbar(im)
cbar.set_label(r'level')
plt.show()

In [ ]:
def crop_map(map_in, factor=1):
    npix = map_in.shape[0]
    map_out = map_in[int(npix/2 - npix/factor):int(npix/2 + npix/factor), int(npix/2 - npix/factor):int(npix/2 + npix/factor)]
    return map_out


# make a dust density map
positions = mdom.gas.xleaf
density   = np.array(mdom.gas.ndust)

# select only cells with non-zero density
ii = density > 1.e-15
pos = np.array((positions[0,:][ii],positions[1,:][ii],positions[2,:][ii]))
lev = levels[ii]
dens = density[ii]

map, _ = ma.make_map(lmax,pos,lev,dens,0.,1,0.,1,0.,1,'z')
deno = np.ones(dens.shape)
weight, _ = ma.make_map(lmax,pos,lev,deno,0.,1,0.,1,0.,1.,'z')

print(weight.shape[0])
weighted_map = map/weight
toto = crop_map(weighted_map, factor=10)
print(toto.shape)

from matplotlib.colors import LogNorm

im = plt.imshow(toto.T,interpolation='nearest',origin='lower', 
                norm=LogNorm(),
                extent=(0.4,0.6,0.4,0.6),
                cmap='gist_earth')

cbar = plt.colorbar(im)
cbar.set_label(r'density')
plt.show()

### 4.3 Run RASCAS 
This is now the RT computation itself. Here, `rascas` (or `rascas-serial`) needs to be run 3 times: once for stellar continuum, once for recombination emission, and once for collisional emission. This may take a bit of time, depending on the number of photon packets each run has to deal with. 

For the dust section, we have to provide the value of the albedo and g at the wavelength of the photons. We can use Table 6 of [Li & Draine (2001)](https://ui.adsabs.harvard.edu/abs/2001ApJ...554..778L/abstract). 

In [ ]:
rootParameters = []  # initialise a list of parameter sections

# H-alpha does not scatter ... use only dust. 
section = 'gas_composition'
params = {
    'nscatterer' : '0',
    'f_ion'      : '1.000E-02',
    'Zref'       : '5.000E-03',
    'ignoreDust' : 'F'
}
rootParameters.append(wpf.param_section(section,params))


section = 'dust'
params = {
    'albedo' : '0.65',
    'g_dust' : '0.5',
    'dust_model' : 'SMC'
}
rootParameters.append(wpf.param_section(section,params))

# RAMSES details
# same as above
rootParameters.append(wpf.param_section(section_ramses,params_ramses))



In [ ]:
# RT for continuum 
Parameters = rootParameters.copy()

section = 'mock'
params = {
'nDirections' : '%i'%nmocks,
'mock_parameter_file' : mock_params_file,
'mock_outputfilename' : 'Cont'
}
Parameters.append(wpf.param_section(section,params))

section = 'rascas'
params = {
    'verbose' : 'T',
    'DomDumpDir' : './',
    'PhotonICFile' : '%s/Cont.IC'%ExperimentDir,
    'fileout' : '%s/Cont.res'%ExperimentDir,
    'nbundle' : '100'
}
Parameters.append(wpf.param_section(section,params))

#### write parameter list to a file 
wpf.write_parameter_file('%s/Cont.cfg'%(ExperimentDir),Parameters)

cmd = "mpirun -n 4 %s/f90/rascas %s/Cont.cfg > %s/rascas_Cont.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file rascas_Cont.log or message below for more info')



In [ ]:
# RT for Collisional emission 
Parameters = rootParameters.copy()

section = 'mock'
params = {
'nDirections' : '%i'%nmocks,
'mock_parameter_file' : mock_params_file,
'mock_outputfilename' : 'ColHa'
}
Parameters.append(wpf.param_section(section,params))

section = 'rascas'
params = {
    'verbose' : 'T',
    'DomDumpDir' : './',
    'PhotonICFile' : '%s/ColHa.IC'%ExperimentDir,
    'fileout' : '%s/ColHa.res'%ExperimentDir,
    'nbundle' : '100'
}
Parameters.append(wpf.param_section(section,params))

#### write parameter list to a file 
wpf.write_parameter_file('%s/ColHa.cfg'%(ExperimentDir),Parameters)

cmd = "mpirun -n 4 %s/f90/rascas %s/ColHa.cfg > %s/rascas_ColHa.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file rascas_ColHa.log or message below for more info')




In [ ]:
# RT for Recombination emission 
Parameters = rootParameters.copy()

section = 'mock'
params = {
'nDirections' : '%i'%nmocks,
'mock_parameter_file' : mock_params_file,
'mock_outputfilename' : 'RecHa'
}
Parameters.append(wpf.param_section(section,params))

section = 'rascas'
params = {
    'verbose' : 'T',
    'DomDumpDir' : './',
    'PhotonICFile' : '%s/RecHa.IC'%ExperimentDir,
    'fileout' : '%s/RecHa.res'%ExperimentDir,
    'nbundle' : '100'
}
Parameters.append(wpf.param_section(section,params))

#### write parameter list to a file 
wpf.write_parameter_file('%s/RecHa.cfg'%(ExperimentDir),Parameters)
cmd = "mpirun -n 4 %s/f90/rascas %s/RecHa.cfg > %s/rascas_RecHa.log"%(RascasDir,ExperimentDir,ExperimentDir)
error = os.system(cmd)
if error != 0:
    print('Something went wrong...')
    print('-> check file rascas_RecHa.log or message below for more info')



## 5. Quick look at the outputs (mocks)

### 5.1 load and show spectra and images in a selected direction 

In [ ]:
idir = np.random.randint(1,nmocks+1) # chose a random mock with idir between 1 and nmocks

# read the three components 
contmock = mockobs(ExperimentDir,'%s/Cont'%ExperimentDir,'%s/Cont.IC'%ExperimentDir,load_spectrum=True,load_image=True,idirection=idir)
recmock  = mockobs(ExperimentDir,'%s/RecHa'%ExperimentDir,'%s/RecHa.IC'%ExperimentDir,load_spectrum=True,load_image=True,idirection=idir)
colmock  = mockobs(ExperimentDir,'%s/ColHa'%ExperimentDir,'%s/ColHa.IC'%ExperimentDir,load_spectrum=True,load_image=True,idirection=idir)

# compute total image and total spectrum 
image = contmock.imtot.copy() + recmock.imtot + colmock.imtot
spec  = contmock.spec.copy() + recmock.spec + colmock.spec

# define x-axis as velocity or wavelength for spectra
x_is_vel = True
if x_is_vel:
    x = v_from_lambda(contmock.spec_lbda_Angstrom)
else: 
    x = contmock.spec_lbda_Angstrom
    
# make the plot : image (left) and spectrum (right)
plt.figure(figsize=(12,5))
plt.subplot(121)
plt.imshow(image.T,norm=LogNorm(vmin=1e36,vmax=1e40))
plt.xticks([]); plt.yticks([])
plt.subplot(122)
plt.plot(x,spec,lw=2,c='k')
plt.xlim(-400,400)
if x_is_vel:
    plt.xlabel('velocity [km/s]',fontsize=13)
else:
    plt.xlabel('wavelength [A]',fontsize=13)
plt.ylabel(r'$F_\lambda\ [erg/s/A]$',fontsize=13)
plt.show()

### 5.2 Write fits files with the data cubes, one per pointing. 

In [ ]:
for idir in range(nmocks):

    contmock = mockobs(ExperimentDir,'%s/Cont'%ExperimentDir,'%s/Cont.IC'%ExperimentDir,load_cube=True,idirection=idir+1)
    recmock  = mockobs(ExperimentDir,'%s/RecHa'%ExperimentDir,'%s/RecHa.IC'%ExperimentDir,load_cube=True,idirection=idir+1)
    colmock  = mockobs(ExperimentDir,'%s/ColHa'%ExperimentDir,'%s/ColHa.IC'%ExperimentDir,load_cube=True,idirection=idir+1)

    cube = contmock.cube + recmock.cube + colmock.cube 
    fits.PrimaryHDU(cube.T).writeto('cube_%i.fits'%(idir),overwrite=True)
